In [22]:
import re
import pandas as pd
import numpy as np

death_metal_df = pd.read_csv("https://raw.githubusercontent.com/orioncohen/metal-bands-by-nation/main/bands.csv")

def operate_on_vec(func):
    def new_func(array):
        list = []
        for i in array:
            list.append(func(i))
        return np.array(list)
    return new_func


def sub_delimiters(genre_or_theme):
    genre_or_theme = re.sub('early', "", genre_or_theme)
    genre_or_theme = re.sub('later', "", genre_or_theme)
    genre_or_theme = re.sub('old', "", genre_or_theme)
    genre_or_theme = re.sub('mid', "", genre_or_theme)
    genre_or_theme = re.sub('[\(\)]', "", genre_or_theme)
    genre_or_theme = re.sub(' - ', " ", genre_or_theme)
    genre_or_theme = re.sub('\d+', " ", genre_or_theme)
    return genre_or_theme.lower()

@operate_on_vec
def genre_to_list(genre):
    genre = sub_delimiters(genre)
    genre_list = re.split('[\|\ \/]', genre)
    genre_list = [genre for genre in genre_list if genre not in ['metal', '', 'n', 'with', 'present']]
    return np.unique(genre_list)

@operate_on_vec
def theme_to_list(theme):
    theme = sub_delimiters(theme)
    genre_list = re.split('[\|\ \/\']', theme)
    genre_list = [genre for genre in genre_list if genre not in ['metal', '', 'n', 'with', '-', ' -', '- ', 'present', '-present']]
    return np.unique(genre_list)

# genre_to_list("Black Metal (early)| Black/Death Metal (later)")
# theme_to_list("Paganism| Indo-European Landscapes| History")
# theme_to_list("Christianity (early| later)| War| Social Issues (mid)")

In [23]:
df = death_metal_df.fillna('')
df = df.assign(genre = lambda df: theme_to_list(df.genre))
df = df.assign(theme = lambda df: theme_to_list(df.theme))

<ipython-input-22-666051a54df4>:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(list)


In [24]:
series = df.genre.apply(pd.Series)
genres_columns = pd.DataFrame(series).iloc[:, :4]
genres_columns.columns = ['genre_1', 'genre_2', 'genre_3', 'genre_4']
series = df.theme.apply(pd.Series)
theme_columns = pd.DataFrame(series).iloc[:, :6]
theme_columns.columns = ['theme_1', 'theme_2', 'theme_3', 'theme_4', 'theme_5', 'theme_6']

In [25]:
big_df = pd.concat([df, genres_columns], axis=1)
big_df = pd.concat([big_df, theme_columns], axis=1)
big_df = big_df.drop(columns=['genre', 'theme'])
big_df

,id,name,country,status,formed_in,active,genre_1,genre_2,genre_3,genre_4,theme_1,theme_2,theme_3,theme_4,theme_5,theme_6
0,1,('M') Inc.,United States,Unknown,2009.0,2009-?,death,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,(sic),United States,Split-up,1993.0,1993-1996,death,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,.F.O.A.D.,France,Active,2009.0,2009-present,death,NaN,NaN,NaN,and,death,life,NaN,NaN,NaN
3,4,100 Suns,United States,Active,2004.0,2004-present,death,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,12 Days of Anarchy,United States,Split-up,1998.0,1998-2002,death,NaN,NaN,NaN,anarchy,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37718,37719,Gladiator,Slovakia,Active,1988.0,1988-present,death,grunge,hard,rock,doom,nuclear,sorrow,war,NaN,NaN
37719,37720,Innersight,Hungary,Active,1997.0,1997-present,ambient,death,electronica,post-rock,NaN,NaN,NaN,NaN,NaN,NaN
37720,37721,Lunarsapian,International,Active,2008.0,2008-present,ambient,black,dark,death,NaN,NaN,NaN,NaN,NaN,NaN
37721,37722,Subconscious,Germany,Active,1990.0,1990-1997| 2002-present,crossover,death,progressive,thrash,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
big_df.to_csv('death_metal_bands.csv')